# Bayes' theorem

Bayesheorem is a formala that describes how to update the probabilities of hypotheses when given evidence.

Given a hypothesis $\theta$ and evidence $x$, Bayes' theorem states that the relationship between the propability of the hypothesis before getting the evidence $P(\theta)$ and the probability of hypothesis after getting the evidence $P(\theta | x)$ is:

\begin{equation}
P(\theta|x)=\frac{P(\theta \cap x)}{P(x)}=\frac{P(x|\theta)· P(\theta)}{P(x)}
\end{equation}

Here we use $P(x|\theta)$ as the likelihood, $P(\theta)$ is the prior, that is the probability that appear $\theta$ in the model and $P(x)$ is the evidence, the probobility of observing the datas given the true model.

In [1]:
#librerias a usar 
import matplotlib. pyplot as plt
import numpy as np
import random
import scipy
import scipy.special
import math
from scipy.optimize import minimize
import scipy.optimize as op
import scipy.stats as stats
import time

%matplotlib inline

Arrangement of the data to work with them:

In [3]:
#datos:
tb1=np.loadtxt('jla_mub_covmatrix.dat')
tb2=np.loadtxt('jla_mub.txt')

# divido los datos de jla_mub.txt en dos arrelgos z y mu 
z=tb2[:,0]
mu=tb2[:,1]

In [4]:
#arreglo los datos de jla_mud_covmatrix.dat en una matriz de 31x31
covmatrix=[]
for i in range(31):
    row=[]
    for j in range(31):
        row.append(tb1[(31*i)+j])
    covmatrix.append(row)

In [5]:
#Pongo las varianzas de la matriz anterior (que se encuentran en la diagunal) en un arreglo.  
var=[]
for i in range(31):
    var.append(covmatrix[i][i])

### Equations:

In [6]:
def sss(omg):
    return pow((1-omg)/omg,(1/3))

In [7]:
def eta(a,omg):
    r=np.sqrt(pow(sss(omg),3)+1)
    return 2*r*pow((pow(a,-4)-(0.1540*sss(omg)*pow(a,-3))+(0.4304*pow(sss(omg),2)*pow(a,-2))+(0.19097*pow(sss(omg),3)*pow(a,-1))+(0.066941*pow(sss(omg),4))),-1/8)

In [8]:
def D_L(z,omg,H):
    return (299792.458/H)*(1+z)*(eta(1,omg)-eta((1/(1+z)),omg))

In [9]:
def mud(z,omg,H):
    return 25+(5*np.log10(D_L(z,omg,H)))-(5*np.log10(H/100))

### Function for the maximum likelihood estimation

In [11]:
def lnlik (datos, P_ini):
    #Likelihood con Parametrso iniciales.
    omg0= P_ini[2]
    H0= P_ini[0] 
    z=datos[0]
    mu=datos[1]
    var=datos[2]
    return np.sum(-0.5*np.divide(pow((mu-mud(z,omg0,H0)),2),var))

In [12]:
def MCMC (datos,P_ini):
    fp=open('Tabla_datos_bayes.dat',"w")
    
    #Parametros iniciales.
    omg0= P_ini[2] #0.4
    H0= P_ini[0] #70.0
    sigma1= P_ini[3] #0.05
    sigma2= P_ini[1] #2
    
    #Likliehood con nuevos doatos.
    for i in range(10000): 
        omg = np.random.normal(P_ini[2],P_ini[3])
        H = np.random.normal(P_ini[0],P_ini[1])
        P_ini2 = np.array([H,P_ini[1],omg,P_ini[3]])
        if omg>0 and H>0:
            
            if lnlik(datos, P_ini2) > lnlik(datos, P_ini):
                P_ini=P_ini2
                fp.write("%f  \t%f \n" % (P_ini[0],P_ini[2]))
            else:
                pro=random.randrange(1,100000000)/100000000
                diff=lnlik2(datos, P_ini2)-lnlik2(datos, P_ini)
                if diff>np.log(pro):
                    P_ini=P_ini2
                    fp.write("%f  \t%f \n" % (P_ini[0],P_ini[2]))
    fp.close()

In [18]:
P_ini = np.array([70,2,0.5,0.05])
datos = np.array([z,mu,var])

In [27]:
nll = lambda *args: -lnlike(*args)
result = op.minimize(nll, [P_ini], args=(datos))
m_ml, b_ml, lnf_ml = result["x"]

NameError: global name 'lnlike' is not defined

Marginalization & uncertainty estimation

In [14]:
def lnprior(P_ini):
    H = P_ini[0]
    omg = P_ini[2]
    if 0 < H < 1 and 50 < omg < 100 :
        return 0.0
    return -np.inf

In [15]:
def lnprob(datos,P_ini):
    lp = lnprior(P_ini)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlik(datos,P_ini)

In [21]:
ndim, nwalkers = 3, 100
pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]

NameError: name 'result' is not defined

In [19]:
import emcee
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(datos, P_ini))

In [20]:
sampler.run_mcmc(pos, 500)

NameError: name 'pos' is not defined